# Evaluation Metrics: SCOT

In [1]:
from shapely.ops import cascaded_union
import matplotlib.pyplot as plt
import geopandas as gpd
import multiprocessing
import pandas as pd
import numpy as np
import skimage.io
import tqdm
import glob
import math
import gdal
import time
import sys
import os
import geopandas as gpd
from shapely import wkt

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
import matplotlib
# matplotlib.use('Agg') # non-interactive

import solaris as sol
from solaris.utils.core import _check_gdf_load
from solaris.raster.image import create_multiband_geotiff 

# import from data_postproc_funcs
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
from sn7_baseline_postproc_funcs import map_wrapper, multithread_polys, \
        calculate_iou, track_footprint_identifiers, \
        sn7_convert_geojsons_to_csv
import solaris.eval.scot as scot

/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Midgard/home/hfang

In [2]:
ground_truth = pd.read_csv('/Midgard/home/hfang/temporal_CD/HRNet_SN7_csv/ground_truth.csv')
aois = [z.split('mosaic_')[-1] for z in ground_truth['filename'].values]
ground_truth['aoi'] = aois
timesteps = [z.split('monthly_')[-1].split('_mosaic')[0] for z in ground_truth['filename'].values]
ground_truth['timestep'] = timesteps
ground_truth = ground_truth.drop('filename', 1)

In [3]:
ground_truth.head()

,id,geometry,aoi,timestep
0,1131,"POLYGON ((1022.739556749351 1023.915693833842,...",L15-0387E-1276N_1549_3087_13,2018_01
1,896,"POLYGON ((839.4174400521442 1023.860588716227,...",L15-0387E-1276N_1549_3087_13,2018_01
2,607,"POLYGON ((977.5393254966475 1021.220368818729,...",L15-0387E-1276N_1549_3087_13,2018_01
3,539,"POLYGON ((804.8580670831725 1021.935737615102,...",L15-0387E-1276N_1549_3087_13,2018_01
4,769,"POLYGON ((817.6027463246137 1022.111423592898,...",L15-0387E-1276N_1549_3087_13,2018_01


In [4]:
pred = pd.read_csv('/Midgard/home/hfang/temporal_CD/HRNet_SN7_csv/submission_hrnet_finetune10.csv')
aois = [z.split('mosaic_')[-1] for z in pred['filename'].values]
pred['aoi'] = aois
timesteps = [z.split('monthly_')[-1].split('_mosaic')[0] for z in pred['filename'].values]
pred['timestep'] = timesteps
pred = pred.drop('filename', 1)

In [5]:
pred.head()

,id,geometry,aoi,timestep
0,0,"POLYGON (( 3 37.3,3 37.7,2.7 37.7,2.7 38.3,2.3...",L15-0632E-0892N_2528_4620_13,2018_01
1,0,"POLYGON (( 3 37.3,3 37.7,2.7 37.7,2.7 38.3,2.3...",L15-0632E-0892N_2528_4620_13,2018_02
2,0,"POLYGON (( 3 37.3,3 37.7,2.7 37.7,2.7 38.3,2.3...",L15-0632E-0892N_2528_4620_13,2018_03
3,0,"POLYGON (( 3 37.3,3 37.7,2.7 37.7,2.7 38.3,2.3...",L15-0632E-0892N_2528_4620_13,2018_04
4,0,"POLYGON (( 3 37.3,3 37.7,2.7 37.7,2.7 38.3,2.3...",L15-0632E-0892N_2528_4620_13,2018_05


In [6]:
aois = sorted(list(ground_truth.aoi.drop_duplicates()))
print(aois)
print(len(aois))

['L15-0387E-1276N_1549_3087_13', 'L15-0566E-1185N_2265_3451_13', 'L15-0632E-0892N_2528_4620_13', 'L15-1015E-1062N_4061_3941_13', 'L15-1200E-0847N_4802_4803_13', 'L15-1276E-1107N_5105_3761_13', 'L15-1438E-1134N_5753_3655_13', 'L15-1615E-1206N_6460_3366_13', 'L15-1690E-1211N_6763_3346_13', 'L15-1848E-0793N_7394_5018_13']
10


In [7]:
cumulative_score = 0.
all_stats = {}

for i, aoi in enumerate(aois):
    print()
    print('%i / %i: AOI %s' % (i + 1, len(aois), aoi))
    grnd_df_one_aoi = ground_truth.loc[ground_truth.aoi == aoi].copy()
    prop_df_one_aoi = pred.loc[pred.aoi == aoi].copy()
    
    grnd_df_one_aoi['geometry'] = grnd_df_one_aoi['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(grnd_df_one_aoi, geometry = 'geometry')
    # print(gdf)
    
    prop_df_one_aoi['geometry'] = prop_df_one_aoi['geometry'].apply(wkt.loads)
    pdf = gpd.GeoDataFrame(prop_df_one_aoi, geometry = 'geometry')
    # print(pdf)

    score_one_aoi, stats_one_aoi = scot.scot_one_aoi(gdf, pdf, threshold=0.25, base_reward=100., beta=2., stats=True, verbose=True)
    cumulative_score += score_one_aoi
    all_stats[aoi] = stats_one_aoi

# Return combined SCOT metric score
score = cumulative_score / len(aois)
print('Overall score: %f' % score)
print(score)
print(all_stats)


1 / 10: AOI L15-0387E-1276N_1549_3087_13
Matching footprints
   1: F1 = 0.4383
   2: F1 = 0.4683
   3: F1 = 0.4701
   4: F1 = 0.4707
   5: F1 = 0.4689
   6: F1 = 0.4691
   7: F1 = 0.4685
   8: F1 = 0.4717
   9: F1 = 0.4746
  10: F1 = 0.4753
  11: F1 = 0.4654
  12: F1 = 0.4612
  13: F1 = 0.4744
  14: F1 = 0.4767
  15: F1 = 0.4762
  16: F1 = 0.4757
  17: F1 = 0.4762
  18: F1 = 0.4776
  19: F1 = 0.4777
  20: F1 = 0.4684
  21: F1 = 0.4654
  22: F1 = 0.4549
Identifying mismatches
Tracking:
    Mismatches: 4
      True Pos: 14069
     False Pos: 8749
     False Neg: 23016
   Track Score: 0.4697
Change Detection:
      True Pos: 165
     False Pos: 304
     False Neg: 342
  Change Score: 0.3381
Combined Score: 0.4358

2 / 10: AOI L15-0566E-1185N_2265_3451_13
Matching footprints
   1: F1 = 0.2899
   2: F1 = 0.2535
   3: F1 = 0.2500
   4: F1 = 0.2500
   5: F1 = 0.3000
   6: F1 = 0.3182
   7: F1 = 0.3191
   8: F1 = 0.3357
   9: F1 = 0.3576
  10: F1 = 0.5141
  11: F1 = 0.5437
  12: F1 = 0.5505
 